## Readme
這個 Notebook 是一個簡易版的 RAG（Retrieval-Augmented Generation）示範系統。

主要功能包含：
-    建立小型知識資料集（自動產生太空小知識並儲存成 CSV）
-    將知識轉換成向量並存入 ChromaDB
-    接收使用者問題，從資料庫搜尋最相關的知識
-    把找到的知識，組成提示語（Prompt），交給 LLM 模型回答
-    支援切換不同的嵌入模型（OpenAI、Chroma、Ollama）與 LLM（GPT-4o、Llama3.2）


🔥 這份檔案能做什麼
- 練習如何將「資料」➔「向量化」➔「建索引」➔「查詢」➔「生成回答」的完整流程
- 學習最基本的 RAG 系統骨架
- 為以後接更大型的知識檢索系統打基礎

### 載入套件、讀取 API 金鑰

In [42]:
import csv
import pandas as pd
import chromadb
from chromadb.utils import embedding_functions
from openai import OpenAI
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")


### 定義 EmbeddingModel 類別

In [43]:
# 這個類別負責決定要用哪種方法把文字轉成向量，你可以選擇
# openai（用 OpenAI 模型轉向量）
# chroma（用 ChromaDB 預設內建的轉向量）
# nomic（用 Ollama 本地模型轉向量）

class EmbeddingModel:
    def __init__(self, model_type="openai"):
        self.model_type = model_type
        if model_type == "openai":
            self.client = OpenAI(api_key=api_key)
            self.embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
                api_key=api_key,
                model_name="text-embedding-3-small",
            )
        elif model_type == "chroma":
            self.embedding_fn = embedding_functions.DefaultEmbeddingFunction()
        elif model_type == "nomic":
            # using Ollama nomic-embed-text model
            self.embedding_fn = embedding_functions.OpenAIEmbeddingFunction(
                api_key="ollama",
                api_base="http://localhost:11434/v1",
                model_name="nomic-embed-text",
            )


### 定義 LLMModel 類別

In [44]:
# 這個類別負責決定用哪個大語言模型 (LLM) 來回答問題，你可以選擇：
#     openai（用 OpenAI GPT）
#     ollama（用本地 Llama 模型）
# 同時也提供一個 generate_completion() 方法 ➔ 把問題丟進去，得到回答。

class LLMModel:
    def __init__(self, model_type="openai"):
        self.model_type = model_type
        if model_type == "openai":
            self.client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
            self.model_name = "gpt-4o-mini"
        else:
            self.client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")
            self.model_name = "llama3.2"

    def generate_completion(self, messages):
        try:
            response = self.client.chat.completions.create(
                model=self.model_name,
                messages=messages,
                temperature=0.0,  # 0.0 is deterministic
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"Error generating response: {str(e)}"

### 選擇模型的介面

In [45]:
# 在 terminal 上讓你自己選要用哪種模型，包括：
#     哪個 LLM（OpenAI 還是 Ollama）
#     哪個向量轉換器（OpenAI / Chroma / Nomic）

def select_models():
    # Select LLM Model
    print("\nSelect LLM Model:")
    print("1. OpenAI GPT-4")
    print("2. Ollama Llama2")
    while True:
        choice = input("Enter choice (1 or 2): ").strip()
        if choice in ["1", "2"]:
            llm_type = "openai" if choice == "1" else "ollama"
            break
        print("Please enter either 1 or 2")

    # Select Embedding Model
    print("\nSelect Embedding Model:")
    print("1. OpenAI Embeddings")
    print("2. Chroma Default")
    print("3. Nomic Embed Text (Ollama)")
    while True:
        choice = input("Enter choice (1, 2, or 3): ").strip()
        if choice in ["1", "2", "3"]:
            embedding_type = {"1": "openai", "2": "chroma", "3": "nomic"}[choice]
            break
        print("Please enter 1, 2, or 3")

    return llm_type, embedding_type

### 自動產生一個小型 CSV
先建立一個資料文件，來模擬要做的資料

In [46]:
import csv

def generate_csv():
    facts = [
        {"id": 1, "fact": "第一位環繞地球飛行的人類是尤里·加加林（1961年）。"},
        {"id": 2, "fact": "阿波羅11號任務於1969年首次將人類送上月球。"},
        {"id": 3, "fact": "哈伯太空望遠鏡於1990年發射，提供了宇宙的驚人影像。"},
        {"id": 4, "fact": "火星是太陽系中被探索最多的行星，美國太空總署已派出多台探測車。"},
        {"id": 5, "fact": "國際太空站（ISS）自2000年11月以來一直持續有人居住。"},
        {"id": 6, "fact": "旅行者1號是目前離地球最遠的人造物體，於1977年發射。"},
        {"id": 7, "fact": "SpaceX由伊隆·馬斯克創立，是第一家將人類送入軌道的私人公司。"},
        {"id": 8, "fact": "詹姆斯·韋伯太空望遠鏡於2021年發射，是哈伯望遠鏡的後繼者。"},
        {"id": 9, "fact": "銀河系包含超過一千億顆恆星。"},
        {"id": 10, "fact": "黑洞是重力極強，連光也無法逃脫的時空區域。"},
    ]

    with open("space_facts.csv", mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=["id", "fact"])
        writer.writeheader()
        writer.writerows(facts)

    print("CSV檔案 'space_facts.csv' 已成功建立！")

generate_csv()


CSV檔案 'space_facts.csv' 已成功建立！


### 載入 CSV

In [47]:
def load_csv():
    df = pd.read_csv("./space_facts.csv")
    documents = df["fact"].tolist()
    # print("載入文件")
    # for doc in documents:
    #     print(f"- {doc}")
    return documents

### 把資料塞進 ChromaDB

In [48]:
def setup_chromadb(documents, embedding_model):
    client = chromadb.Client()
    try:
        client.delete_collection("space_facts")
    except:
        pass
    collection = client.create_collection(
        name="space_facts", embedding_function=embedding_model.embedding_fn
    )
    collection.add(documents=documents, ids=[str(i) for i in range(len(documents))])
    print("文件已成功加入 ChromaDB ！\n")
    return collection

### 根據問題，找相關的小段資料
當你問一個問題

去 ChromaDB 資料庫找最接近問題意思的小知識

回傳找出來的資料。

In [49]:
def find_related_chunks(query, collection, top_k=2):
    results = collection.query(query_texts=[query], n_results=top_k)

    print("\nRelated chunks found:")
    for doc in results["documents"][0]:
        print(f"- {doc}")

    return list(
        zip(
            results["documents"][0],
            (
                results["metadatas"][0]
                if results["metadatas"][0]
                else [{}] * len(results["documents"][0])
            ),
        )
    )

### 整理 Prompt

In [50]:
def augment_prompt(query, related_chunks):
    context = "\n".join([chunk[0] for chunk in related_chunks])
    augmented_prompt = f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"

    print("\nAugmented prompt: ⤵️")
    print(augmented_prompt)

    return augmented_prompt

### 整個 RAG 流程

In [51]:
def rag_pipeline(query, collection, llm_model, top_k=2):
    print(f"\nProcessing query: {query}")

    related_chunks = find_related_chunks(query, collection, top_k)
    augmented_prompt = augment_prompt(query, related_chunks)

    response = llm_model.generate_completion(
        [
            {
                "role": "system",
                "content": "You are a helpful assistant who can answer questions about space but only answers questions that are directly related to the sources/documents given.",
            },
            {"role": "user", "content": augmented_prompt},
        ]
    )

    print("\nGenerated response:")
    print(response)

    references = [chunk[0] for chunk in related_chunks]
    return response, references


In [ ]:
print("Starting the RAG pipeline demo...")

# Select models
llm_type, embedding_type = select_models()

# Initialize models
llm_model = LLMModel(llm_type)
embedding_model = EmbeddingModel(embedding_type)

print(f"\nUsing LLM: {llm_type.upper()}")
print(f"Using Embeddings: {embedding_type.upper()}")

# Generate and load data
# generate_csv()
documents = load_csv()

# Setup ChromaDB
collection = setup_chromadb(documents, embedding_model)

# Run queries
queries = [
    "哈勃太空望遠鏡是什麼？",
    "跟我說說火星探索",
]

for query in queries:
    print("\n" + "=" * 50)
    print(f"Processing query: {query}")
    response, references = rag_pipeline(query, collection, llm_model)

    print("\nFinal Results:")
    print("-" * 30)
    print("Response:", response)
    print("\nReferences used:")
    for ref in references:
        print(f"- {ref}")
    print("=" * 50)

Starting the RAG pipeline demo...

Select LLM Model:
1. OpenAI GPT-4
2. Ollama Llama2

Select Embedding Model:
1. OpenAI Embeddings
2. Chroma Default
3. Nomic Embed Text (Ollama)

Using LLM: OLLAMA
Using Embeddings: CHROMA
文件已成功加入 ChromaDB ！


Processing query: What is the Hubble Space Telescope?

Processing query: What is the Hubble Space Telescope?

Related chunks found:
- 銀河系包含超過一千億顆恆星。
- 火星是太陽系中被探索最多的行星，美國太空總署已派出多台探測車。

Augmented prompt: ⤵️
Context:
銀河系包含超過一千億顆恆星。
火星是太陽系中被探索最多的行星，美國太空總署已派出多台探測車。

Question: What is the Hubble Space Telescope?
Answer:

Generated response:
Unfortunately, I don't have any information about the Hubble Space Telescope from the provided context. The text only mentions the number of stars in a galaxy and the exploration of Mars by NASA.

However, if you'd like to provide more context or information, I can try to help you answer your question about the Hubble Space Telescope.

Final Results:
------------------------------
Response: Unfortunately, I don't ha